In [1]:
import pandas as pd

In [2]:
FEATURES_QUALI = 'features_quali_complete_P3.csv'

# PREDICT IN Q3 OR NOT

In [3]:
quali_features = pd.read_csv(FEATURES_QUALI, index_col=False)

In [4]:
quali_features['in_q3'] = quali_features['Quali_Rank'].apply(lambda x: 1 if x <= 10 else 0)

In [5]:
def map_compound(row):
    if row['Compound'] == 'SOFT':
        return row['Soft']
    elif row['Compound'] == 'MEDIUM':
        return row['Medium']
    elif row['Compound'] == 'HARD':
        return row['Hard']
    else:
        return None
    
quali_features['Compound'] = quali_features.apply(map_compound, axis=1)

compound_map = {'C6' : 6, 'C5' : 5, 'C4' : 4, 'C3' : 3, 'C2' : 2, 'C1' : 1} # wet = 0
quali_features['Compound'] = quali_features['Compound'].map(compound_map).fillna(0)

In [6]:
drop_columns = ['Time', 'DriverNumber', 'Soft', 'Medium', 'Hard', 'Country', 'EventDate',
                'LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time', 'Quali_Rank',
                'LapNumber', 'TyreLife', 'FreshTyre', 'Stint']

quali_features = quali_features.drop(columns=drop_columns)

In [7]:
quali_features_encoded = features_encoded = pd.get_dummies(quali_features, columns=['Driver','Team','Location', 'Year'], drop_first=False)

In [8]:
quali_features

,Driver,Team,Compound,AirTemp,TrackTemp,Rainfall,Humidity,WindSpeed,WindDirection,TrackLenght,...,Peak_Downforce_N,Avg_Downforce_N,Peak_Drag_N,Peak_Front_Load_N,Peak_Rear_Load_N,Team_Pace_EMA,Driver_Teammate_Gap_EMA,Driver_Consistency_EMA,Degradation_EMA,in_q3
0,LEC,Ferrari,3.0,19.4,24.3,False,51.0,0.6,343,5.412,...,30061.176736,15110.574496,8124.642361,15967.697325,23405.277478,-1.606421,-0.418709,1.294945,0.138100,1
1,VER,Red Bull Racing,3.0,19.4,24.2,False,51.0,0.8,2,5.412,...,30822.189082,14709.982510,8330.321373,15830.695983,23927.015267,-1.454666,-0.214258,1.176590,0.131253,1
2,SAI,Ferrari,3.0,19.3,24.5,False,53.0,0.5,354,5.412,...,29872.410002,14717.819360,8073.624325,15139.076301,23294.986842,-1.606421,0.418709,1.285464,0.088300,1
3,PER,Red Bull Racing,3.0,19.4,24.2,False,51.0,0.8,2,5.412,...,31013.928520,14899.578713,8382.142843,16030.054736,24011.176938,-1.454666,0.214258,1.278144,0.114015,1
4,HAM,Mercedes,3.0,19.3,25.0,False,55.0,0.6,10,5.412,...,29872.410002,14982.631982,8073.624325,14955.892800,23263.427286,-0.951462,-0.340234,1.172743,0.100051,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1598,COL,Alpine,4.0,34.2,46.5,False,35.0,1.0,83,5.513,...,30822.189082,14341.431708,8330.321373,16022.029189,23903.996256,0.809274,0.023433,1.185255,-0.037586,0
1599,BOR,Kick Sauber,4.0,34.0,46.5,False,36.0,1.7,198,5.513,...,31013.928520,14398.443736,8382.142843,15790.023815,24004.686968,0.304365,0.300834,1.148753,-0.009591,0
1600,OCO,Haas F1 Team,4.0,34.2,46.5,False,35.0,1.0,83,5.513,...,31206.262500,14320.249434,8434.125000,15701.891150,24134.756086,0.429606,0.458798,1.294687,-0.027581,0
1601,STR,Aston Martin,4.0,33.8,46.1,False,37.0,1.6,285,5.513,...,31013.928520,14305.130511,8382.142843,15360.052484,24036.061922,0.294133,0.250399,1.204013,-0.036862,0


In [9]:
features_location_split = pd.read_csv(FEATURES_QUALI, index_col=False)

circuits = pd.read_csv('f1_unique_circuits_complete.csv', delimiter=',')

first_few_events_2025 = circuits['Location'][0:10]

train_mask = (
    (features_location_split['Year'] < 2025) |
    ((features_location_split['Year'] == 2025) & features_location_split['Location'].isin(first_few_events_2025))
)
test_mask = (
    (features_location_split['Year'] == 2025) &
    (~features_location_split['Location'].isin(first_few_events_2025))
)

"""train_idx = features_location_split[train_mask].index
test_idx = features_location_split[test_mask].index

X_train = quali_features_encoded.loc[train_idx].drop(columns=['in_q3'])
y_train = quali_features_encoded.loc[train_idx, 'in_q3']

X_test = quali_features_encoded.loc[test_idx].drop(columns=['in_q3'])
y_test = quali_features_encoded.loc[test_idx, 'in_q3']"""

# Prendiamo l'intero set di dati X (tutte le feature)
X = quali_features_encoded.drop(columns=['in_q3'])
# Prendiamo l'intero set di dati y (il nostro target)
y = quali_features_encoded['in_q3']

# --- 3. Applicare le maschere per creare i set di Train e Test ---
X_train = X[train_mask]
y_train = y[train_mask]

X_test = X[test_mask]
y_test = y[test_mask]

In [10]:
from sklearn.preprocessing import StandardScaler

numerical_cols_float = X_train.select_dtypes(include=['float64']).columns
numerical_cols_int = X_train.select_dtypes(include=['int64']).columns

cols_to_scale = list(numerical_cols_float) + list(numerical_cols_int)

scaler = StandardScaler()

X_train_scaled = X_train.copy()
X_train_scaled[cols_to_scale] = scaler.fit_transform(X_train[cols_to_scale])

X_test_scaled = X_test.copy()
X_test_scaled[cols_to_scale] = scaler.transform(X_test[cols_to_scale])


In [22]:
X_train_scaled

,Compound,AirTemp,TrackTemp,Rainfall,Humidity,WindSpeed,WindDirection,TrackLenght,NumTurns,Traction,...,Location_Spa-Francorchamps,Location_Spielberg,Location_Suzuka,Location_São Paulo,Location_Yas Island,Location_Zandvoort,Year_2022,Year_2023,Year_2024,Year_2025
0,-0.767032,-0.791822,-0.957673,False,-0.196288,-1.060802,1.486506,0.331987,-0.532559,0.832174,...,False,False,False,False,False,False,True,False,False,False
1,-0.767032,-0.791822,-0.968732,False,-0.196288,-0.876510,-1.710740,0.331987,-0.532559,0.832174,...,False,False,False,False,False,False,True,False,False,False
2,-0.767032,-0.810026,-0.935554,False,-0.096010,-1.152947,1.589643,0.331987,-0.532559,0.832174,...,False,False,False,False,False,False,True,False,False,False
3,-0.767032,-0.791822,-0.968732,False,-0.196288,-0.876510,-1.710740,0.331987,-0.532559,0.832174,...,False,False,False,False,False,False,True,False,False,False
4,-0.767032,-0.810026,-0.880257,False,0.004269,-1.060802,-1.635731,0.331987,-0.532559,0.832174,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1428,1.151218,-0.427757,1.121504,False,-1.249209,0.321385,-1.570099,-0.938913,-0.816895,0.832174,...,False,False,False,False,False,False,False,False,False,True
1429,1.151218,-0.427757,1.121504,False,-1.249209,0.321385,-1.570099,-0.938913,-0.816895,0.832174,...,False,False,False,False,False,False,False,False,False,True
1430,1.151218,-0.445961,1.176801,False,-1.098792,0.782114,-1.579475,-0.938913,-0.816895,0.832174,...,False,False,False,False,False,False,False,False,False,True
1431,1.151218,-0.427757,1.143623,False,-1.199070,0.321385,-1.345073,-0.938913,-0.816895,0.832174,...,False,False,False,False,False,False,False,False,False,True


## Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

log_reg = LogisticRegression(max_iter=1000, random_state=42)

log_reg.fit(X_train_scaled, y_train)

y_pred_log_reg = log_reg.predict(X_test_scaled)

accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)

In [12]:
print(f"\nAccuratezza (Logistic Regression): {accuracy_log_reg * 100:.2f}%")
print("\nClassification Report (Logistic Regression):")
# 'target_names' ci aiuta a leggere il report più facilmente
print(classification_report(y_test, y_pred_log_reg, target_names=['No Q3 (0)', 'In Q3 (1)']))


Accuratezza (Logistic Regression): 80.00%

Classification Report (Logistic Regression):
              precision    recall  f1-score   support

   No Q3 (0)       0.75      0.88      0.81        83
   In Q3 (1)       0.86      0.72      0.79        87

    accuracy                           0.80       170
   macro avg       0.81      0.80      0.80       170
weighted avg       0.81      0.80      0.80       170



## Random Forest Classifier

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

rf_model.fit(X_train_scaled, y_train)

y_pred_rf = rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)

In [14]:
print(f"\nAccuratezza (Random Forest): {accuracy_rf * 100:.2f}%")
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=['No Q3 (0)', 'In Q3 (1)']))


Accuratezza (Random Forest): 76.47%

Classification Report (Random Forest):
              precision    recall  f1-score   support

   No Q3 (0)       0.73      0.83      0.78        83
   In Q3 (1)       0.81      0.70      0.75        87

    accuracy                           0.76       170
   macro avg       0.77      0.77      0.76       170
weighted avg       0.77      0.76      0.76       170



In [25]:
X_train_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1433 entries, 0 to 1432
Columns: 101 entries, Compound to Year_2025
dtypes: bool(70), float64(31)
memory usage: 456.2 KB


In [16]:
if not isinstance(X_train_scaled, pd.DataFrame):
    # Se X_train_scaled è un array numpy, recuperiamo i nomi da X_train
    feature_names = X_train.columns
else:
    feature_names = X_train_scaled.columns

importances_rf = rf_model.feature_importances_

rf_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances_rf
})

rf_importance_df = rf_importance_df.sort_values(by='Importance', ascending=False)

rf_importance_df.head(15)

,Feature,Importance
28,Team_Pace_EMA,0.187487
29,Driver_Teammate_Gap_EMA,0.048495
2,TrackTemp,0.029015
30,Driver_Consistency_EMA,0.028569
18,Avg_Lap_Speed,0.027987
21,Peak_Braking_G,0.027598
31,Degradation_EMA,0.026372
5,WindSpeed,0.026175
20,Peak_Cornering_G,0.025537
19,Min_Corner_Speed,0.025129


Questo modello ci dice quali feature ha "consultato" più spesso.

    Team_Pace_EMA (Importanza: 0.187): Questa è la tua feature numero 1 indiscussa. Da sola, rappresenta quasi il 19% del potere decisionale del modello! Conferma che il passo gara (Race Pace) è il singolo miglior predittore della performance in qualifica.

    Driver_Teammate_Gap_EMA (Importanza: 0.048): La tua seconda feature personalizzata più importante.

    TrackTemp, Driver_Consistency_EMA, Avg_Lap_Speed: Il modello usa anche le condizioni della pista e le tue metriche di consistenza e velocità.

    Peak_Braking_G (Importanza: 0.027): La tua prima feature di fisica pura entra in classifica.

Conclusione (Random Forest): Il modello si basa principalmente sulle tue feature EMA e sulle feature fisiche.

In [17]:
coefficients_log_reg = log_reg.coef_[0]

# 2. Crea un DataFrame per visualizzarli
log_reg_coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients_log_reg
})

# 3. Ordina per valore assoluto per vedere l'impatto maggiore (sia positivo che negativo)
log_reg_coef_df['Abs_Coefficient'] = abs(log_reg_coef_df['Coefficient'])
log_reg_coef_df = log_reg_coef_df.sort_values(by='Abs_Coefficient', ascending=False)

log_reg_coef_df.drop(columns=['Abs_Coefficient']).head(15)

,Feature,Coefficient
33,Driver_ALO,1.544818
24,Avg_Downforce_N,1.473815
28,Team_Pace_EMA,-1.462224
84,Location_Miami,1.379287
80,Location_Lusail,-1.317761
2,TrackTemp,-1.246469
60,Driver_VER,1.189864
95,Location_Yas Island,-1.172824
17,Max_Straight_Speed,-1.168954
78,Location_Jeddah,-1.129245


I "Coefficienti" ci dicono come una feature influenza il risultato.

    Positivo (+): Aumenta la probabilità di andare in Q3 (y=1).

    Negativo (-): Aumenta la probabilità di essere eliminati (y=0).

Top Predittori di Q3 (Coefficienti Positivi)

    Driver_ALO (+1.54) / Driver_VER (+1.18): Questo è "l'effetto Alonso/Verstappen". Il modello ha imparato che, a parità di altre condizioni, se il pilota è Alonso o Verstappen, la probabilità di entrare in Q3 è molto più alta. Questo conferma la tua intuizione di usare il one-hot encoding per Driver!

    Avg_Downforce_N (+1.47): Un'intuizione fisica fantastica! Il modello ha imparato che le auto con un carico aerodinamico medio più alto (più grip) hanno più probabilità di entrare in Q3.

    Location_Miami (+1.37): Una stranezza statistica, probabilmente dovuta al fatto che team/piloti specifici (come Alonso, che è anche in cima) hanno sovraperformato a Miami.

Top Predittori di Eliminazione (Coefficienti Negativi)

    Team_Pace_EMA (-1.46): Questa è la conferma perfetta. Ricorda che un passo gara veloce è un numero negativo (es. -0.5s più veloce della media). Il modello ha imparato che un Team_Pace_EMA negativo predice fortemente l'ingresso in Q3. Ha funzionato!

    TrackTemp (-1.24): Una temperatura della pista più alta diminuisce le possibilità di Q3. Questo è un insight reale: con la pista calda, è più difficile gestire le gomme.

    Max_Straight_Speed (-1.16): Questo è l'insight più bello. Una velocità di punta più alta predice una eliminazione dalla Q3. Questo conferma la teoria secondo cui i team più scarsi (es. Haas, Williams) spesso usano assetti a basso carico (bassa resistenza aerodinamica) per essere veloci sui rettilinei, ma questo li penalizza così tanto in curva da farli eliminare.

    Driver_DEV (-1.10) / Team_Kick Sauber (-1.02): Il modello ha imparato correttamente che De Vries e la Kick Sauber (Alfa Romeo) erano forti predittori di eliminazione.

1. L'Obiettivo: Trovare "l'Ordine di Grandezza"

L'obiettivo di una prima ricerca (come RandomizedSearchCV) non è trovare il valore esatto (es. n_estimators=412), ma trovare la "zona" giusta. Stiamo cercando di "mettere tra parentesi" (bracket) il valore migliore.

2. Spiegazione dei Parametri (Perché quei valori?)

Usiamo la griglia della Random Forest come esempio:

 - n_estimators: [200, 400, 800] (Numero di alberi)

    Cosa fa: Controlla la potenza del modello. Più alberi = più potente (di solito), ma più lento.

    Perché quei valori: Stiamo testando se "più è meglio". Iniziamo con 200 (già più del default di 100). Se il modello migliore fosse 800, il prossimo passo sarebbe provare 1200 o 1500. Se il modello migliore fosse 200, potremmo scoprire che 150 è sufficiente. Stiamo testando scale diverse.

 - max_depth: [10, 20, 30, None] (Profondità massima dell'albero)

    Cosa fa: Questo è il controllo principale contro l'overfitting. None (il default) fa sì che l'albero cresca all'infinito, imparando "a memoria" i dati di training.

    Perché quei valori: Stiamo testando il trade-off tra complessità e generalizzazione.

        10 = Un albero "basso", veloce, semplice. Sarà accurato?

        30 = Un albero "molto profondo". Rischia l'overfitting?

        None = Il rischio massimo di overfitting.

        Stiamo cercando il "punto debole" (sweet spot) in cui l'albero è abbastanza profondo da imparare, ma non così profondo da imparare a memoria.

 - min_samples_split: [2, 5, 10] (Campioni minimi per dividere un nodo)

    Cosa fa: Un altro controllo sull'overfitting. Il default è 2.

    Perché quei valori: Stiamo testando se "costringere" l'albero ad essere più generale aiuta. Aumentando a 5 o 10, diciamo al modello: "Non disturbarti a creare una nuova regola se non hai almeno 5 o 10 campioni che la supportano". Questo previene la creazione di regole basate su un singolo outlier.

 - min_samples_leaf: [1, 2, 4] (Campioni minimi in una foglia finale)

    Cosa fa: Simile a min_samples_split, ma per il risultato finale.

    Perché quei valori: Il default è 1. Stiamo testando 2 e 4 per vedere se il modello migliora quando le sue previsioni finali sono basate su una media di (almeno) 2 o 4 campioni, invece che su uno solo.

 - max_features: ['sqrt', 'log2', 0.5] (Quante feature usare per ogni albero)

    Cosa fa: Controlla la "casualità" della foresta.

    Perché quei valori: sqrt (radice quadrata del n° di feature) e log2 sono i due valori standard usati in letteratura. Ho aggiunto 0.5 (usa il 50% delle feature) solo come test aggiuntivo per vedere se un approccio diverso funziona meglio.

La Strategia in Sintesi

    Non si sceglie a caso: Si parte dai valori di default e si prova "un po' di più", "un po' di meno" o "un ordine di grandezza diverso".

    Si usano valori comuni: Molti di questi (sqrt, 2, 1) sono semplicemente i valori standard o di default.

    Si usa RandomizedSearchCV (Ricerca Casuale): È il modo migliore per "esplorare" velocemente questo enorme spazio di parametri. Prova 50-100 combinazioni casuali e ci dice quale "zona" è la più promettente.

    Si usa GridSearchCV (Ricerca a Griglia): Dopo aver trovato la zona migliore (es. max_depth è intorno a 10), si può fare una ricerca a griglia ristretta (es. [8, 10, 12]) per trovare il valore esatto.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

rf_dist_model = RandomForestClassifier(random_state=42, n_jobs=-1)

param_dist = {
    'n_estimators' : [200, 400, 800, 1000],
    'max_depth' : [10, 20, 30 , None],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    'max_features' : ['sqrt', 'log2', 0.5]
}

random_search = RandomizedSearchCV(
    estimator=rf_dist_model,
    param_distributions=param_dist,
    n_iter=50,
    cv=3,
    scoring='f1',
    verbose=2,
    random_state=42,
    n_jobs=-1  # Usa tutti i core
)

random_search.fit(X_train_scaled, y_train)

# Best model
best_rf = random_search.best_estimator_

Fitting 3 folds for each of 50 candidates, totalling 150 fits


 RandomForestClassifier(max_features=0.5, min_samples_leaf=4,

min_samples_split=10, n_estimators=200, n_jobs=-1,

random_state=42) 

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

rf_grid_model = RandomForestClassifier(random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators' : [200, 400, 800],
    'max_depth' : [10, 20, 30 , None],
    'min_samples_split' : [2, 5, 10],
    'min_samples_leaf' : [1, 2, 4],
    'max_features' : ['sqrt', 'log2', 0.5]
}

# Grid search with 3-fold cross-validation
grid_search = GridSearchCV(
    estimator=rf_grid_model,
    param_grid=param_grid,
    cv=3,
    scoring='f1',
    verbose=2
)

grid_search.fit(X_train_scaled, y_train)

# Best model
best_rf = grid_search.best_estimator_

Fitting 3 folds for each of 324 candidates, totalling 972 fits
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.2s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=400; total time=   0.4s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=800; total time=   0.9s
[CV] END max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=8

In [23]:
best_rf

RandomForestClassifier(max_depth=20, min_samples_split=5, n_estimators=200,
                       n_jobs=-1, random_state=42)

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report


best_rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth = 20,
    min_samples_split = 5,
    min_samples_leaf = 1,
    max_features = 'sqrt',
    random_state=42,
    n_jobs=-1
    )

best_rf_model.fit(X_train_scaled, y_train)

y_pred_rf = best_rf_model.predict(X_test_scaled)

accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"\nAccuratezza (Random Forest): {accuracy_rf * 100:.2f}%")
print("\nClassification Report (Random Forest):")
print(classification_report(y_test, y_pred_rf, target_names=['No Q3 (0)', 'In Q3 (1)']))


Accuratezza (Random Forest): 78.24%

Classification Report (Random Forest):
              precision    recall  f1-score   support

   No Q3 (0)       0.74      0.84      0.79        83
   In Q3 (1)       0.83      0.72      0.77        87

    accuracy                           0.78       170
   macro avg       0.79      0.78      0.78       170
weighted avg       0.79      0.78      0.78       170



In [32]:
importances_rf = best_rf_model.feature_importances_

rf_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances_rf
})

rf_importance_df = rf_importance_df.sort_values(by='Importance', ascending=False)

rf_importance_df.head(15)

,Feature,Importance
28,Team_Pace_EMA,0.202633
29,Driver_Teammate_Gap_EMA,0.052070
65,Team_Ferrari,0.027426
18,Avg_Lap_Speed,0.026552
2,TrackTemp,0.025840
30,Driver_Consistency_EMA,0.025730
67,Team_Kick Sauber,0.025487
21,Peak_Braking_G,0.025167
5,WindSpeed,0.024272
6,WindDirection,0.024229


# OLD CODE

In [2]:
FILE_2025 = 'features_laps_2025.csv'
FILE_2024 = 'features_laps_2024.csv'
FILE_2023 = 'features_laps_2023.csv'
FILE_2022 = 'features_laps_2022.csv'


features_25 = pd.read_csv(FILE_2025, delimiter=',')
features_24 = pd.read_csv(FILE_2024, delimiter=',')
features_23 = pd.read_csv(FILE_2023, delimiter=',')
features_22 = pd.read_csv(FILE_2022, delimiter=',')

features = pd.concat([features_25, features_24, features_23, features_22], ignore_index=True)

remove_2025 = ['Austin', 'Mexico City', 'São Paulo', 'Las Vegas', 'Lusail', 'Yas Island']
features = features[~((features['Year'] == 2025) & (features['Location'].isin(remove_2025)))]

features.to_csv('features_laps_p3.csv', index=False)

In [3]:
FEATURES_FILE = "features_laps_p3.csv"
features = pd.read_csv(FEATURES_FILE, delimiter=',')

In [4]:
features['Year'].value_counts()

Year
2023    2309
2024    2285
2022    2151
2025    1679
Name: count, dtype: int64

In [5]:
features['Driver'].value_counts()

Driver
LEC    536
HAM    528
RUS    525
VER    509
SAI    500
NOR    487
ALO    446
GAS    402
OCO    397
TSU    386
ALB    379
PIA    377
PER    359
STR    358
HUL    297
BOT    288
MAG    272
ZHO    265
RIC    207
LAW    116
SAR    111
MSC     91
ANT     90
HAD     89
BEA     78
BOR     73
VET     71
COL     61
LAT     60
DEV     42
DOO     24
Name: count, dtype: int64

## Encoding

In [7]:
group_cols = ['Year', 'Location', 'Driver']

features_encoded = (features.sort_values('LapTime').groupby(group_cols, as_index=False).first())
features_location_split = features_encoded.copy()

In [8]:
features_encoded['Driver'].value_counts()

Driver
ALO    88
RUS    88
VER    88
NOR    87
SAI    87
LEC    87
HAM    86
ALB    85
STR    84
TSU    84
GAS    83
OCO    83
PER    67
BOT    67
ZHO    66
PIA    66
MAG    65
HUL    64
RIC    47
SAR    34
LAW    29
MSC    21
LAT    20
COL    20
VET    20
BEA    19
ANT    18
HAD    18
BOR    18
DEV    10
DOO     7
Name: count, dtype: int64

In [9]:
features_encoded = pd.get_dummies(features_encoded, columns=['Driver','Team','Location'], drop_first=False)

In [10]:
def map_compound(row):
    if row['Compound'] == 'SOFT':
        return row['Soft']
    elif row['Compound'] == 'MEDIUM':
        return row['Medium']
    elif row['Compound'] == 'HARD':
        return row['Hard']
    else:
        return None
    
features_encoded['Compound'] = features_encoded.apply(map_compound, axis=1)

compound_map = {'C6' : 6, 'C5' : 5, 'C4' : 4, 'C3' : 3, 'C2' : 2, 'C1' : 1} # wet = 0
features_encoded['Compound'] = features_encoded['Compound'].map(compound_map).fillna(0)

In [11]:
features_encoded = features_encoded.drop(columns=['Country','EventDate','EventName','Soft','Medium','Hard'])

In [12]:
# fixing Nan values for used tyres
features_encoded.loc[features_encoded['TyreLife'].isna() & (features_encoded['FreshTyre'] == True), 'TyreLife'] = 2.0
features_encoded.loc[features_encoded['TyreLife'].isna() & (features_encoded['FreshTyre'] == False), 'TyreLife'] = 3.0

In [13]:
features_encoded = features_encoded.drop(columns=['RaceDistance', 'NumLaps', 'LapNumber',
                                                  'TyreLife', 'FreshTyre', 'Stint',
                                                  'Sector1Time', 'Sector2Time', 'Sector3Time'])

In [14]:
features_encoded.to_csv('features_laps_encoded_p3.csv', index=False)

In [15]:
FEATURES_ENCODED_FILE = "features_laps_encoded_p3.csv"
features_encoded = pd.read_csv(FEATURES_ENCODED_FILE, delimiter=',')

In [16]:
features_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706 entries, 0 to 1705
Data columns (total 83 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Year                        1706 non-null   int64  
 1   LapTime                     1706 non-null   float64
 2   Compound                    1706 non-null   float64
 3   TrackTemp                   1706 non-null   float64
 4   AirTemp                     1706 non-null   float64
 5   TrackLenght                 1706 non-null   float64
 6   NumTurns                    1706 non-null   int64  
 7   Taction                     1706 non-null   int64  
 8   AsphaltGrip                 1706 non-null   int64  
 9   AsphaltAbrasion             1706 non-null   int64  
 10  TrackEvolution              1706 non-null   int64  
 11  TyreStress                  1706 non-null   int64  
 12  Braking                     1706 non-null   int64  
 13  LateralLoad                 1706 

## Training the model

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

X = features_encoded.drop(columns=['LapTime'])
y = features_encoded['LapTime']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"MAE: {mae:.3f} seconds")
print(f"RMSE: {rmse:.3f} seconds")

MAE: 0.431 seconds
RMSE: 1.012 seconds


In [23]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

gbr = GradientBoostingRegressor(random_state=42)

param_grid = {
    'n_estimators': [200, 400, 800],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 3],
    'max_features': [None, 0.8, 'sqrt']
}

grid = GridSearchCV(
    gbr, 
    param_grid, 
    scoring='neg_mean_absolute_error', 
    cv=3, 
    n_jobs=-1, 
    verbose=2
    )

grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best MAE:", -grid.best_score_)

Fitting 3 folds for each of 324 candidates, totalling 972 fits
Best params: {'learning_rate': 0.05, 'max_depth': 7, 'max_features': 0.8, 'min_samples_leaf': 3, 'min_samples_split': 2, 'n_estimators': 800}
Best MAE: 0.40442137805641815


In [22]:
from sklearn.ensemble import GradientBoostingRegressor
 
best_gbr = GradientBoostingRegressor(
    n_estimators=800, 
    learning_rate=0.05, 
    max_depth=7,
    max_features=0.8,
    min_samples_leaf=3,
    min_samples_split=2,
    random_state=42
    )

best_gbr.fit(X_train, y_train)
preds_gbr = best_gbr.predict(X_test)

mae = mean_absolute_error(y_test, preds_gbr)
rmse = np.sqrt(mean_squared_error(y_test, preds_gbr))
print(f"MAE: {mae:.3f} seconds")
print(f"RMSE: {rmse:.3f} seconds")

MAE: 0.373 seconds
RMSE: 1.032 seconds


In [30]:
circuits = pd.read_csv('f1_unique_circuits_complete.csv', delimiter=',')

first_few_events_2025 = circuits['Location'][0:10]

In [31]:
first_few_events_2025

0    Melbourne
1     Shanghai
2       Suzuka
3       Sakhir
4       Jeddah
5        Miami
6        Imola
7       Monaco
8    Barcelona
9     Montréal
Name: Location, dtype: object

In [32]:
train_mask = (
    (features_location_split['Year'] < 2025) |
    ((features_location_split['Year'] == 2025) & features_location_split['Location'].isin(first_few_events_2025))
)
test_mask = (
    (features_location_split['Year'] == 2025) &
    (~features_location_split['Location'].isin(first_few_events_2025))
)

train_idx = features_location_split[train_mask].index
test_idx = features_location_split[test_mask].index

X_train = features_encoded.loc[train_idx].drop(columns=['LapTime'])
y_train = features_encoded.loc[train_idx, 'LapTime']

X_test = features_encoded.loc[test_idx].drop(columns=['LapTime'])
y_test = features_encoded.loc[test_idx, 'LapTime']

In [35]:
from sklearn.ensemble import GradientBoostingRegressor
 
best_gbr = GradientBoostingRegressor(
    n_estimators=800, 
    learning_rate=0.05, 
    max_depth=7,
    max_features=0.8,
    min_samples_leaf=3,
    min_samples_split=2,
    random_state=42
    )

best_gbr.fit(X_train, y_train)
preds_gbr = best_gbr.predict(X_test)

mae = mean_absolute_error(y_test, preds_gbr)
rmse = np.sqrt(mean_squared_error(y_test, preds_gbr))
print(f"MAE: {mae:.3f} seconds")
print(f"RMSE: {rmse:.3f} seconds")

NameError: name 'X_train' is not defined

In [20]:
X_train

,Year,Compound,TrackTemp,AirTemp,TrackLenght,NumTurns,Taction,AsphaltGrip,AsphaltAbrasion,TrackEvolution,...,Location_Monza,Location_Sakhir,Location_Shanghai,Location_Silverstone,Location_Spa-Francorchamps,Location_Spielberg,Location_Suzuka,Location_São Paulo,Location_Yas Island,Location_Zandvoort
0,2022,4.0,35.4,29.9,5.513,20,4,3,3,4,...,False,False,False,False,False,False,False,False,False,False
1,2022,4.0,33.4,29.3,5.513,20,4,3,3,4,...,False,False,False,False,False,False,False,False,False,False
2,2022,4.0,33.4,29.3,5.513,20,4,3,3,4,...,False,False,False,False,False,False,False,False,False,False
3,2022,4.0,36.7,30.6,5.513,20,4,3,3,4,...,False,False,False,False,False,False,False,False,False,False
4,2022,4.0,34.0,29.4,5.513,20,4,3,3,4,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1682,2025,3.0,29.5,14.5,5.807,18,3,4,4,3,...,False,False,False,False,False,False,True,False,False,False
1683,2025,3.0,30.6,14.6,5.807,18,3,4,4,3,...,False,False,False,False,False,False,True,False,False,False
1684,2025,3.0,33.7,14.7,5.807,18,3,4,4,3,...,False,False,False,False,False,False,True,False,False,False
1685,2025,3.0,33.7,14.9,5.807,18,3,4,4,3,...,False,False,False,False,False,False,True,False,False,False


In [29]:
y_pred

array([ 84.55289,  82.98865, 101.79341, 102.09829, 101.1839 ,  91.71674,
        72.91264,  77.74745,  90.52327, 101.99858,  71.7367 , 107.6929 ,
        70.34227,  88.07923,  76.25203,  64.88269, 102.16126, 105.17609,
        77.19564, 102.82082,  86.51141,  88.75086,  72.3912 ,  79.23575,
        65.37907,  91.11892,  92.08934,  78.79706, 106.31169,  94.07677,
        82.99159,  83.2516 ,  90.50012,  66.6553 , 102.80741,  71.77241,
        65.35041,  90.30679,  92.38309,  77.64554,  71.04551,  75.85711,
        65.3742 ,  88.05024,  80.67657,  83.08602,  89.926  ,  90.04894,
        66.10457,  89.63409,  83.34135, 102.06653,  72.80861,  79.95425,
        89.65083,  87.66606,  77.97916,  88.35653,  82.56416,  79.65562,
        71.89484,  91.14497,  71.60348,  91.35939,  93.97254,  86.81627,
        71.90782,  72.28482,  88.11514,  83.70541,  78.65135,  91.56559,
        65.55883,  77.79079,  80.86677,  77.78155,  89.51127,  71.63625,
        88.80124,  77.75422,  91.54082,  65.15942, 

In [28]:
pred_df = X_test.copy()  # copy the features used for prediction
pred_df['ActualLapTime'] = y_test.values
pred_df['PredictedLapTime'] = y_pred
pred_df['Driver'] = features.loc[X_test.index, 'Driver']
pred_df['Team'] = features.loc[X_test.index, 'Team']
pred_df['Location'] = features.loc[X_test.index, 'Location']


cols_to_keep = ['Driver', 'Team', 'LapNumber', 'Stint', 'Compound', 
                'PredictedLapTime', 'ActualLapTime', 'Location', 'Year']
pred_df = pred_df[cols_to_keep]

ValueError: Length of values (342) does not match length of index (258)